## EU Taxonomy 2

In [ ]:
# load imports
import pandas as pd
import numpy as np
import json, math
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, HTML
# set dataframe options
pd.set_option('display.max_columns', None)

In [ ]:
# execute the helper functions defined for accessing RDP REST API calls 
%run RDPDefines.ipynb

### Load the pre-downloaded database for EU Taxonomy

In [ ]:
orgs = pd.read_pickle('OrganizationClassification.pkl').astype({
	'AlignedFlag': object,
	'AsReportedFlag': bool,
	'CurrencyId': int,
	'CurrencyIdISO': object,
	'CurrencyIdDESC': object,
	'FinancialPeriodId': object,
	'GreenRevenueSourcedFlag': object,
	'MinimumSocialSafeguardComment': object,
	'MinimumSocialSafeguardPassedFlag': object,
	'OrganizationId': np.int64,
	'PeriodFiscalYear': int,
	'TotalCapexAmount': float,
	'TotalEligibleAlignedCapexAmount': float,
	'TotalEligibleAlignedCapexPercent': float,
	'TotalEligibleAlignedOpexAmount': float,
	'TotalEligibleAlignedOpexPercent': float,
	'TotalEligibleAlignedRevenueAmount': float,
	'TotalEligibleAlignedRevenuePercent': float,
	'TotalEligibleCapexAmount': float,
	'TotalEligibleCapexPercent': float,
	'TotalEligibleNotAlignedCapexAmount': float,
	'TotalEligibleNotAlignedCapexPercent': float,
	'TotalEligibleNotAlignedOpexAmount': float,
	'TotalEligibleNotAlignedOpexPercent': float,
	'TotalEligibleNotAlignedRevenueAmount': float,
	'TotalEligibleNotAlignedRevenuePercent': float,
	'TotalEligibleOpexAmount': float,
	'TotalEligibleOpexPercent': float,
	'TotalEligibleRevenueAmount': float,
	'TotalEligibleRevenuePercent': float,
	'TotalNotEligibleCapexAmount': float,
	'TotalNotEligibleCapexPercent': float,
	'TotalNotEligibleOpexAmount': float,
	'TotalNotEligibleOpexPercent': float,
	'TotalNotEligibleRevenueAmount': float,
	'TotalNotEligibleRevenuePercent': float,
	'TotalOpexAmount': float,
	'TotalRevenueAmount': float
})
orgs.head()

In [ ]:
# load the MSS dataset
mss = pd.read_pickle('SocialSafeguards.pkl').astype({
	'OrganizationId': np.int64,
	'MSS-9': bool,
	'MSS-5': bool,
	'MSS-6': bool,
	'MSS-7': bool,
	'MSS-8': bool,
	'MSS-1': bool,
	'MSS-2': bool,
	'MSS-11': bool,
	'MSS-3': bool,
	'MSS-10': bool,
	'MSS-4': bool,
})
mss.head()

### Load the equity portfolio to be analized
Portfolio is a CSV file with the constituent *RIC* and corresponding *Weight*
|RIC|WEIGHT|
|--|--|
|AAPL.O|0.23|
|MSFT.O|0.21|
|..|..|

In [ ]:
# load the portfolio to be analized
portfolio = pd.read_csv('Portfolio.csv')
portfolio

Lookup *Organization PermID* and *Reporting currency* for all instruments

RDP SYmbology API: [/data/symbology/beta1/convert]()

In [ ]:
allResponses = []
RIClist = portfolio['RIC'].tolist()
bucketSize = 90

buckets = [ RIClist[i: i + bucketSize] for i in range(0, len(RIClist), bucketSize) ]
for bucket in buckets:
  reqData = {
    "universe": bucket,
    "to": ["OrganizationId"]
  }

  hResp = postRequest('/data/symbology/beta1/convert', reqData)
  print(hResp)
  allResponses = allResponses + hResp['universe']

In [ ]:
# merge Symbology and Currency data into the portfolio
orgsData = pd.DataFrame(allResponses)
portfolio = portfolio.merge(orgsData, how='inner', left_on='RIC', right_on='Instrument')
portfolio = portfolio.drop(['Instrument'], axis = 1)
portfolio['Organization PermID'] = portfolio['Organization PermID'].astype(np.int64)
portfolio

In [ ]:
# merge the taxonomy data into the portfolio
temp = portfolio.merge(mss, how='left', left_on='Organization PermID', right_on='OrganizationId')
temp.drop('OrganizationId', axis=1, inplace=True)
mPortfolio = temp.merge(orgs, how='left', left_on='Organization PermID', right_on='OrganizationId')
latestFYear = mPortfolio['PeriodFiscalYear'].max()
mPortfolio['PeriodFiscalYear'].fillna(latestFYear, inplace=True)
mPortfolio.head()

In [ ]:
# summarize the data for the latest FY
df1 = mPortfolio[mPortfolio['PeriodFiscalYear'] == latestFYear]
df1.reset_index(drop=True, inplace=True)

### Draw the stats for the whole portfolio for latest Fiscal Year

In [ ]:
# function below sets the color based on amount
def SetColor(x):
    if(x > 69):
        return "green"
    elif(x < 30):
        return "red"
    else:
        return "darkgoldenrod"

def generateIndicator(value, total, header, row, col):
    vPer = (value / total) * 100
    return go.Indicator(
        mode = "gauge+number",
        value = vPer,
        domain = {'row': row, 'column': col},
        title = {'text': header },
        gauge = { 'axis': {'range': [None, 100]},
                'bar': {'color': SetColor(vPer)},
                })    

# plot the data for latest FY
totalInstruments = len(df1)
TaxoCovered = len(df1['OrganizationId'].dropna())
alignedFlag = df1['AlignedFlag'].sum()
greenRevenueFlag = df1['GreenRevenueSourcedFlag'].sum()
ssgPassedFlag = df1['MinimumSocialSafeguardPassedFlag'].sum()

fig = go.Figure()
fig.add_trace(generateIndicator(TaxoCovered, totalInstruments, 'Coverage % ({}/{} have Taxonomy data)'.format(TaxoCovered, totalInstruments), 0, 1))
fig.add_trace(generateIndicator(alignedFlag, totalInstruments, 'Aligned %', 1, 0))
fig.add_trace(generateIndicator(greenRevenueFlag, totalInstruments, 'Green Revenue %', 1, 1))
fig.add_trace(generateIndicator(ssgPassedFlag, totalInstruments, 'Minimum Social Safeguard Passed %', 1, 2))
fig.update_layout(grid = {'rows': 2, 'columns': 3, 'pattern': "independent"}, title_text="Portfolio Ratios")
fig.show()


# TODO: !!! Important: Convert the revenue amounts to a single currency -> USD !!!


# Scale the revenue amounts according to portfolio weights
eligibleAlignedRev = (df1['TotalEligibleAlignedRevenueAmount'] * df1['Weight']).sum()
eligibleNotAlignedRev = (df1['TotalEligibleNotAlignedRevenueAmount'] * df1['Weight']).sum()
totalEligibleRev = (df1['TotalEligibleRevenueAmount'] * df1['Weight']).sum()
totalNotEligibleRev = (df1['TotalNotEligibleRevenueAmount'] * df1['Weight']).sum()
totalRev = (df1['TotalRevenueAmount'] * df1['Weight']).sum()

fig = go.Figure()
fig.add_trace(go.Indicator(
        mode = "number",
        value = totalRev,
        domain = {'row': 0, 'column': 0},
        title = {'text': 'Total Revenue' },
        gauge = { 'axis': {'range': [None, 100]}}))

fig.add_trace(generateIndicator(eligibleAlignedRev, totalRev, 'Eligible Aligned Revenue %', 1, 1))
fig.add_trace(generateIndicator(eligibleNotAlignedRev, totalRev, 'Eligible Not-Aligned Revenue %', 1, 2))
fig.add_trace(generateIndicator(totalEligibleRev, totalRev, 'Eligible Revenue %', 0, 1))
fig.add_trace(generateIndicator(totalNotEligibleRev, totalRev, 'Not-Eligible Revenue %', 0, 2))
fig.update_layout(grid = {'rows': 2, 'columns': 3, 'pattern': "independent"}, title_text="Portfolio Revenue")
fig.show()

### Show the Taxonomy alignment for individual holdings

In [ ]:
# filtered = pd.DataFrame()
output = widgets.Output()

ricsDD = widgets.Dropdown(options = sorted(mPortfolio['RIC'].unique()), description='RIC:')
fyDD = widgets.Dropdown(options = sorted(mPortfolio['PeriodFiscalYear'].unique().astype(int)), description='FY:')

def filterData(newRIC, newFY):
    # global filtered
    # output.clear_output()
    filtered = mPortfolio[(mPortfolio['RIC'] == newRIC) & (mPortfolio['PeriodFiscalYear'] == newFY)]
    with output:
        if len(filtered) == 0:
            output.clear_output()
            display(HTML('<div style="font-size: 40px;">No Data</div>'))
        else:    
            tRow = filtered.iloc[0]
            def fmt1(ky):
                return '{0:,.0f} M'.format(tRow[ky]/1000000)

            def fmt2(ky1, ky2):
                return '{0:,.0f} M'.format(tRow[ky1]/1000000) + ' ({0:.2g} %)'.format(tRow[ky2])

            fHtml = '<div id="myDiv"><div style="font-size: 50px; color: #0000E0;">' + tRow['Company Common Name'] + '</div><table><tr><td><div style="font-size: 30px;">' + tRow['RIC'] + '</div><div style="font-size: 20px; color: #707070;">RIC</div></td><td><div style="font-size: 30px;">' + tRow['Reporting Currency'] + '</div><div style="font-size: 20px; color: #707070;">Currency</div></td><td><span style="font-size: 20px; color: #707070;">Is Aligned</span></td><td><span style="font-size: 20px;">' + str(tRow['AlignedFlag']) + '</span></td></tr><tr><td><div style="font-size: 30px;">' + str(tRow['Organization PermID']) + '</div><div style="font-size: 20px; color: #707070;">Org PermID</div></td><td><div style="font-size: 30px;">' + '{0:.0f}'.format(tRow['PeriodFiscalYear']) + '</div><div style="font-size: 20px; color: #707070;">Fiscal Year</div></td><td><span style="font-size: 20px; color: #707070;">As Reported</span></td><td><span style="font-size: 20px;">' + str(tRow['AsReportedFlag']) + '</span></td></tr><tr><td><div style="font-size: 30px;">' + '{0:.2g} %'.format(tRow['Weight']) + '</div><div style="font-size: 20px; color: #707070;">Weight</div></td><td><div style="font-size: 30px;">' + fmt1('TotalRevenueAmount') + '</div><div style="font-size: 20px; color: #707070;">Total Revenue</div></td><td><span style="font-size: 20px; color: #707070;">Is Green Revenue</span></td><td><span style="font-size: 20px;">' + str(tRow['GreenRevenueSourcedFlag']) + '</span></td></tr><tr><td>&nbsp;</td><td><div style="font-size: 30px;">' + fmt1('TotalCapexAmount') + '</div><div style="font-size: 20px; color: #707070;">Total Capex</div></td><td><span style="font-size: 20px; color: #707070;">MSS Passed</span></td><td><span style="font-size: 20px;">' + str(tRow['MinimumSocialSafeguardPassedFlag']) + '</span></td></tr><tr><td/><td><div style="font-size: 30px;">' + fmt1('TotalOpexAmount') + '</div><div style="font-size: 20px; color: #707070;">Total Opex</div></td><td><span style="font-size: 20px; color: #707070;">MSS Comment</span></td><td><span style="font-size: 20px;">' + str(tRow['MinimumSocialSafeguardComment']) + '</span></td></tr></table><table><tr><td><table><tr><td><span style="font-size: 30px; color: #000070;">Revenue</span></td><td><span style="font-size: 20px;">&nbsp;</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Eligible Aligned Capex</span></td><td><span style="font-size: 20px;">' + fmt2('TotalEligibleAlignedCapexAmount','TotalEligibleAlignedCapexPercent') + '</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Eligible Aligned Opex</span></td><td><span style="font-size: 20px;">' + fmt2('TotalEligibleAlignedOpexAmount','TotalEligibleAlignedOpexPercent') + '</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Eligible Aligned Revenue</span></td><td><span style="font-size: 20px;">' + fmt2('TotalEligibleAlignedRevenueAmount','TotalEligibleAlignedRevenuePercent') + '</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Eligible Capex</span></td><td><span style="font-size: 20px;">' + fmt2('TotalEligibleCapexAmount','TotalEligibleCapexPercent') + '</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Eligible NotAligned Capex</span></td><td><span style="font-size: 20px;">' + fmt2('TotalEligibleNotAlignedCapexAmount','TotalEligibleNotAlignedCapexPercent') + '</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Eligible NotAligned Opex</span></td><td><span style="font-size: 20px;">' + fmt2('TotalEligibleNotAlignedOpexAmount','TotalEligibleNotAlignedOpexPercent') + '</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Eligible NotAligned Revenue</span></td><td><span style="font-size: 20px;">' + fmt2('TotalEligibleNotAlignedRevenueAmount','TotalEligibleNotAlignedRevenuePercent') + '</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Eligible Opex</span></td><td><span style="font-size: 20px;">' + fmt2('TotalEligibleOpexAmount','TotalEligibleOpexPercent') + '</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Eligible Revenue</span></td><td><span style="font-size: 20px;">' + fmt2('TotalEligibleRevenueAmount','TotalEligibleRevenuePercent') + '</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Not Eligible Capex</span></td><td><span style="font-size: 20px;">' + fmt2('TotalNotEligibleCapexAmount','TotalNotEligibleCapexPercent') + '</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Not Eligible Opex</span></td><td><span style="font-size: 20px;">' + fmt2('TotalNotEligibleOpexAmount','TotalNotEligibleOpexPercent') + '</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Not Eligible Revenue</span></td><td><span style="font-size: 20px;">' + fmt2('TotalNotEligibleRevenueAmount', 'TotalNotEligibleRevenuePercent') + '</span></td></tr></table></td><td><table><tr><td><span style="font-size: 30px; color: #000070;">Social Safeguards</span></td><td><span style="font-size: 20px;">&nbsp;</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Human Rights Policy</span></td><td><span style="font-size: 20px;">' + str(tRow['MSS-1']) + '</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Child Labor Policy</span></td><td><span style="font-size: 20px;">' + str(tRow['MSS-2']) + '</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Forced Labor Policy</span></td><td><span style="font-size: 20px;">' + str(tRow['MSS-3']) + '</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Diversity Policy</span></td><td><span style="font-size: 20px;">' + str(tRow['MSS-4']) + '</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Freedom of Association Policy</span></td><td><span style="font-size: 20px;">' + str(tRow['MSS-5']) + '</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Bribery and Corruption Policy</span></td><td><span style="font-size: 20px;">' + str(tRow['MSS-6']) + '</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Whistleblower Protection</span></td><td><span style="font-size: 20px;">' + str(tRow['MSS-7']) + '</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Fundamental Human Rights</span></td><td><span style="font-size: 20px;">' + str(tRow['MSS-8']) + '</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Health and Safety Policy</span></td><td><span style="font-size: 20px;">' + str(tRow['MSS-9']) + '</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Diversity and Oppertunity</span></td><td><span style="font-size: 20px;">' + str(tRow['MSS-10']) + '</span></td></tr><tr><td><span style="font-size: 20px; color: #707070;">Business Ethics Policy</span></td><td><span style="font-size: 20px;">' + str(tRow['MSS-11']) + '</span></td></tr></table></td></tr></table></div>'
            output.clear_output()
            display(HTML(fHtml))

def ricsDDEH(change):
    filterData(change.new, fyDD.value)

def fyDDEH(change):
    filterData(ricsDD.value, change.new)

ricsDD.observe(ricsDDEH, names='value')
fyDD.observe(fyDDEH, names='value')
input_widgets = widgets.HBox([ricsDD, fyDD])
display(input_widgets)
display(output)

### ONE TIME PROCESS - Download the bulk JSON files

In [ ]:
# define the download function
def downloadJSONBulkFile(bucketName, fileAttributes, fileNameKeywords):
    # get a list of all the buckets
    hResp = getRequest('/file-store/v1/file-sets?bucket=' + bucketName + '&pageSize=100&attributes=' + fileAttributes)
    print(hResp)
    # loop through all the buckets
    for bucket in hResp['value']:
        bName = bucket['name']
        # does bucket contains all the matching keywords
        if all([x in bName for x in fileNameKeywords]):
            fileName = bucket['files'][0]
            print('Found bucket: ', bName, ', FileName: ', fileName)
            # stop any more searching
            break
    
    if not fileName:
        raise Exception('No matching bulk file found in bucket:'.format(bucketName))

    # download and uncompress the file object
    fileStr = downloadUncompressFile('/file-store/v1/files/' + fileName + '/stream')
    print('File downloaded and uncompressed, size: ', len(fileStr))
    return fileStr


#### Download and save the Bulk EU Taxonomy file

In [ ]:
# download the EU Taxonomy classification bulk file
jsonlFile = downloadJSONBulkFile('Bulk-EuTax', 'ContentType:EuTax Global OrganizationClassification Full', ['Full', 'Json', 'Init'])
# parse out the entries in the file
orgData = []
for l in jsonlFile.splitlines():
    jObj = json.loads(l)
    orgData.append(jObj['EUTaxonomyOrganizationClassification'])
print('Loaded {} organizations'.format(len(orgData)))
    

In [ ]:
# load the dataset into a pandas dataframe
df1 = pd.DataFrame(orgData)
# save the database
df1.to_pickle('OrganizationClassification.pkl')

### Other EU Taxonomy bulk files available are:
* EuTax Global SegmentClassification Full
* EuTax Global ObjectiveClassification Full
* EuTax Global PassedSocialSafeguard Full
* EuTax Global GreenRevenueBreakout Full
* EuTax Global Financial Segment Full
* EuTax Global Activity Full
* EuTax Global SegmentRevenueBreakdown Full
* EuTax Global OrganizationClassification Full



In [ ]:
# download the Mininum Social Safeguards bulk file
jsonlFile = downloadJSONBulkFile('Bulk-EuTax', 'ContentType:EuTax Global PassedSocialSafeguard Full', ['Full', 'Json', 'Init'])

flags = []
for l in jsonlFile.splitlines():
    jObj = json.loads(l)
    d = {}
    d['OrganizationId'] = jObj['ObjectId'].split(';', 1)[0]
    for flag in jObj['PassedSocialSafeguard']['PassedSocialSafeguards']:
        d[flag['TechnicalScreeningCriteriaId']] = flag['PassedFlag']
    flags.append(d)

In [ ]:
# load the dataset into a pandas dataframe
df2 = pd.DataFrame(flags)
# save into a pickle
df2.to_pickle('SocialSafeguards.pkl')